# 네이버 영화평 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

- 데이터 가져오기: '네이버 영화리뷰 데이터'로 검색

In [8]:
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')
train_df.shape, test_df.shape 

((150000, 3), (50000, 3))

In [3]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리

- 트레인 데이터 셋

In [9]:
# Null 데이터가 있는지 확인
train_df.document.isna().sum()

5

In [12]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(149995, 3)

In [13]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [14]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [15]:
# 긍정/부정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [16]:
# Null 데이터 확인
test_df.document.isna().sum()

3

In [17]:
# Null 데이터 제거
test_df.dropna(how='any', inplace=True)
test_df.shape

(49997, 3)

In [18]:
# 중복 확인
test_df.document.nunique()

49157

In [19]:
# 중복 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [20]:
# Label 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리

- train dataset

In [22]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [23]:
review = 'What a good movie!!!'
review1 = '                    '
review1.strip()

''

In [24]:
# '' 만 남은 데이터 --> np.nan 으로 변경시킨 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [25]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- test dataset

In [26]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

- 전처리가 끝난 데이터는 저장

In [27]:
train_df.to_csv('data/naver_movie_train_전처리완료.csv', sep='\t', index=False)
test_df.to_csv('data/naver_movie_test_전처리완료.csv', sep='\t', index=False)

### 3. 한글 처리 - SOYNLP

In [28]:
import joblib
scores = joblib.load('data/scores.pkl')

In [29]:
# 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
from soynlp.tokenizer import MaxScoreTokenizer
max_tokenizer = MaxScoreTokenizer(scores)

In [30]:
# 형태소로 tokenize
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
max_tokenizer.tokenize(text)

['교도소', '이야기', '구먼', '솔직', '히', '재미는', '없다', '평점', '조정']

In [31]:
# Stopword 출처: https://mr-doosun.tistory.com/24
stop_text = '은 는 이 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'
stop_list = stop_text.split()

In [32]:
word_list = max_tokenizer.tokenize(text)
word_list

['교도소', '이야기', '구먼', '솔직', '히', '재미는', '없다', '평점', '조정']

In [33]:
word_list = [word for word in word_list if word not in stop_list]
word_list

['교도소', '이야기', '구먼', '솔직', '히', '재미는', '평점', '조정']

In [34]:
review = ' '.join(word_list)
review

'교도소 이야기 구먼 솔직 히 재미는 평점 조정'

In [35]:
X_train = []
for document in train_df.document[:3]:
    word_list = max_tokenizer.tokenize(document)
    word_list = [word for word in word_list if word not in stop_list]
    tokenized_str = ' '.join(word_list)
    X_train.append(tokenized_str)

In [36]:
X_train

['더빙 진짜 짜증 나네요 목소리', '흠 포스터 보고 초딩 영화 오버 연기 가볍지 않구나', '너무 재밓었다 보는 것을 추천 한다']

In [37]:
train_df.document[:3]

0                  아 더빙   진짜 짜증나네요 목소리
1    흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

- X_train, X_test, y_train, y_test 만들기

In [38]:
from tqdm.notebook import tqdm
X_train = []
for document in tqdm(train_df.document):
    word_list = max_tokenizer.tokenize(document)
    word_list = [word for word in word_list if word not in stop_list]
    tokenized_str = ' '.join(word_list)
    X_train.append(tokenized_str)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [39]:
X_test = []
for document in tqdm(test_df.document):
    word_list = max_tokenizer.tokenize(document)
    word_list = [word for word in word_list if word not in stop_list]
    tokenized_str = ' '.join(word_list)
    X_test.append(tokenized_str)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [40]:
y_train = train_df.label.values
y_test = test_df.label.values

### 4. Feature 변환 - CountVectorizer

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [42]:
X_train_cv.shape, X_test_cv.shape 

((145393, 159131), (48852, 159131))

### 5. 모델 생성/학습/평가

In [43]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2022)
lrc.fit(X_train_cv, y_train)
lrc.score(X_test_cv, y_test)

0.823569147629575

### 6. 실제 테스트

In [44]:
review1 = '오락성으로만 평가하긴엔 역사적배경에 킹스맨의 배경까지 의미있고 필수적인 내용이 있었다고 생각합니다. 4도 기대해볼만 하네요. 제발 킹스맨 1,2 팬들이라면 믿고 보러가세요. 그래야 이 명작 시리즈가 안끊겨요!'
review2 = '매튜 본 감독이 킹스맨 1이 성공한 이유를 아직까지 모른다는 루머가 사실인 것 같다.'

In [45]:
# 전처리
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review1).strip()
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', review2).strip()

In [46]:
# 형태소 분석 및 불용어 처리
words = max_tokenizer.tokenize(review1)
words = [word for word in words if word not in stop_list]
review1 = ' '.join(words)
review1

'오락성 평가 하긴엔 역사 배경에 킹스 맨의 배경 의미 있고 필수적인 내용이 있었다 고 합니다 도 기 볼만 하네요 제발 킹스 맨 팬들 이라 면 믿고 보러가세요 그래야 명작 시리즈 안끊겨요'

In [47]:
words = max_tokenizer.tokenize(review2)
words = [word for word in words if word not in stop_list]
review2 = ' '.join(words)
review2

'매튜 본 감독 킹스 맨 성공 이유를 아직 모른다 는 루머가'

In [48]:
# Feature 변환
review = [review1, review2]
review_cv = cvect.transform(review)

In [49]:
# 예측
lrc.predict(review_cv)

array([1, 0], dtype=int64)

### 7. 하이퍼 파라메터 튜닝

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [51]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lrc', LogisticRegression(random_state=2022))
])
params = {
    'cvect__ngram_range': [(1,1), (1,2)],
    'cvect__max_df': [0.9, 0.99],
    'cvect__min_df': [1, 3],
    'lrc__C': [1, 5]
}

In [52]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, scoring='accuracy', cv=3
)
%time grid_pipe.fit(X_train, y_train)

Wall time: 14min 8s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect', CountVectorizer()),
                                       ('lrc',
                                        LogisticRegression(random_state=2022))]),
             param_grid={'cvect__max_df': [0.9, 0.99], 'cvect__min_df': [1, 3],
                         'cvect__ngram_range': [(1, 1), (1, 2)],
                         'lrc__C': [1, 5]},
             scoring='accuracy')

In [53]:
grid_pipe.best_params_

{'cvect__max_df': 0.9,
 'cvect__min_df': 1,
 'cvect__ngram_range': (1, 2),
 'lrc__C': 1}

In [54]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8313272742159994

### 8. 나이브 베이즈 모델

In [55]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [56]:
cvect2 = CountVectorizer(min_df=1, max_df=0.9, ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((145393, 873554), (48852, 873554))

In [57]:
nb.fit(X_train_cv2, y_train)
nb.score(X_test_cv2, y_test)

0.8420740194874314